In [1]:
import simsopt
import matplotlib
from pathlib import Path
import numpy as np
from simsopt.field import DipoleField, ToroidalField
from simsopt.geo import PermanentMagnetGrid, SurfaceRZFourier
from simsopt.solve import GPMO
from simsopt.util.permanent_magnet_helper_functions import *

In [2]:
nphi = 16
ntheta = 16
filename = "simsopt/tests/test_files/wout_c09r00_fixedBoundary_0.5T_vacuum_ns201.nc"
s = SurfaceRZFourier.from_wout(filename, range="half period", nphi=nphi, ntheta=ntheta)

In [3]:
net_poloidal_current_Amperes = 3.7713e+6
mu0 = 4 * np.pi * 1e-7
RB = mu0 * net_poloidal_current_Amperes / (2 * np.pi)
bs = ToroidalField(R0=1, B0=RB)
bs.set_points(s.gamma().reshape((-1, 3)))
Bnormal = np.sum(bs.B().reshape((nphi, ntheta, 3)) * s.unitnormal(), axis=2)

In [6]:
kwargs = {"coordinate_flag": "cylindrical"}
famus_filename = 'simsopt/tests/test_files/init_orient_pm_nonorm_5E4_q4_dp.focus'
pm_opt = PermanentMagnetGrid.geo_setup_from_famus(
    s, Bnormal, famus_filename, **kwargs
)


f_B (total with initial SIMSOPT guess) =  0.21659773300989732


In [9]:
kwargs = initialize_default_kwargs('GPMO')
kwargs['K'] = 40000
kwargs['nhistory'] = 100

# Optimize the permanent magnets greedily
R2_history, Bn_history, m_history = GPMO(pm_opt, **kwargs)

Number of binary dipoles to use in GPMO algorithm =  400
Number of binary dipoles returned by GPMO algorithm =  400


In [10]:
b_dipole = DipoleField(
  pm_opt.dipole_grid_xyz,
  pm_opt.m,
  nfp=s.nfp,
  coordinate_flag=pm_opt.coordinate_flag,
  m_maxima=pm_opt.m_maxima,
)
b_dipole.set_points(s.gamma().reshape((-1, 3)))
Bnormal_dipoles = np.sum(b_dipole.B().reshape((nphi, ntheta, 3)) * s.unitnormal(), axis=-1)
Bnormal_total = Bnormal + Bnormal_dipoles

# Now save vtk files of Bnormal, Bnormal_dipoles, and Bnormal_total for Paraview viewing
pointData = {"B_N": Bnormal[:, :, None]}
s.to_vtk("CoilsBn", extra_data=pointData)
pointData = {"B_N": Bnormal_dipoles[:, :, None]}
s.to_vtk("MagnetsBn", extra_data=pointData)
pointData = {"B_N": Bnormal_total[:, :, None]}
s.to_vtk("TotalBn", extra_data=pointData)